In [ ]:
pdfFile = '/Catalogues/Catalogue Spanish_2.pdf'

# <span style="color:yellow">**Page image preprocessing**</span>

## preview the bboxes in pdf page --> get_text() <span style="color:red">AND get_links()</span>
# This is just to see the original test data in the glossay pae (i am going to keep it commented out)

In [ ]:
# import fitz
# from PIL import Image
# import IPython.display as display

# # Open the document
# doc = fitz.open(pdfFile)#'redacted.pdf')

# # Select a page
# page = doc[8]#11]

# # Get the bboxes
# bboxes = []#[fitz.Rect(50, 50, 100, 100), fitz.Rect(150, 150, 200, 200)]#[fitz.Rect(141, 200, 1005, 565)]

# # print(page.get_links())
# print(page.get_text('dict')['blocks'])
# for block in page.get_text('dict')['blocks']:#for block in page.get_links():
#     bboxes.append((block['bbox']))


# # Draw a rectangle on the page for each bbox
# for bbox in bboxes:
#     page.draw_rect(bbox, color=(1, 0, 0))#, fill=(1, 0, 0), width=0.3, overlay=True)

# # Render the page to an image
# pix = page.get_pixmap()

# # # Save the image
# # pix.save('preview.png')


# # Get the image data from the pixmap
# data = pix.samples

# # Create a PIL image from the data
# img = Image.frombytes("RGB", [pix.width, pix.height], data)

# # Display the image
# display.display(img)

## remove all anotations (text and or images) from page --> Page.get_text()

In [ ]:
import fitz
from PIL import Image
import IPython.display as display

#------------------------------------------------------------------------------------
# 1 -\\\\\\\\\\\- Remove Anotations (text, and images) -\\\\\\\\\\\\\\\\\\\\\\\\\\\\\
#------------------------------------------------------------------------------------
def remove_annotations(page, extractictionType='text'):
    ''' Requires a page object from fitz.open() 
                            and the type of extraction ('text', 'image' or 'both')'''
    # Get the list of blocks
    blocks = page.get_text('dict')['blocks']

    # Count the number of image blocks
    num_image_blocks = sum(1 for block in blocks if 'image' in block)

    # Count the number of text blocks
    num_text_blocks = sum(1 for block in blocks if 'lines' in block)

    if extractictionType == 'text':
        if num_text_blocks == 0:
            return page
        elif num_image_blocks == 0:
            threshold = 2
        elif num_image_blocks > 0:
            threshold = num_text_blocks - num_image_blocks
    elif extractictionType == 'image' :
        if num_image_blocks == 0:
            return page
        elif num_text_blocks == 0:
            threshold = 0
            page_bbox = fitz.Rect(page.rect)
            page.add_redact_annot(page_bbox)
            page.apply_redactions()
        elif num_text_blocks > 0:
            # if num_image_blocks > num_text_blocks:
            #     threshold = num_image_blocks - num_text_blocks
            # threshold = abs(num_image_blocks - num_text_blocks)
            """ for this to work i would need to create an incremental threshold, uppon each iteration the threshold would increase by 1, and the while loop would stop when the threshold is reached """
    elif extractictionType == 'both':
        threshold = 0
        page_bbox = fitz.Rect(page.rect)
        page.add_redact_annot(page_bbox)
        page.apply_redactions()
    else:
        return page

    # Remove all other annotations from the page
    while len(page.get_text('dict')['blocks'])>threshold:
        for block in page.get_text('dict')['blocks']:
            if 'lines' in block and extractictionType == 'text' or 'lines' in block and extractictionType == 'both':
                for line in block['lines']:
                    for span in line['spans']:
                        searchText = span['text']
                        # Search for the text
                        text_instances = page.search_for(searchText)
                        # Redact the text
                        for inst in text_instances:
                            page.add_redact_annot(inst)
                            page.apply_redactions()
            elif 'image' in block and extractictionType == 'image' or 'image' in block and extractictionType == 'both':
                # If the block is an image, redact it
                image_bbox = fitz.Rect(block['bbox'])
                page.add_redact_annot(image_bbox)
                page.apply_redactions()
        # print(page.get_text('dict')['blocks'])
        # page.draw_rect(block['bbox'], color=(1, 0, 0))#, fill=(1, 0, 0), width=0.3, overlay=True)
    # # Save the redacted PDF
    # doc.save('redacted_fullText.pdf', garbage=4, deflate=True, clean=True)
    return page

#------------------------------------------------------------------------------------
# \\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\
# ------------------------------------------------------------------------------------

## get the <b>bboxes of the images</b> in the 'infographic glossary'
<span style="color:yellow"><b>note:</b> This is displaying the resulting bboxes in a pop up window. <b style="color:red">It needs to be closed for the next cells to be run</b></span>

In [ ]:
import cv2
import fitz
import numpy as np


# Function to merge close rectangles
def merge_close_rectangles(bboxes, threshold):
    # Function to calculate distance between two rectangles
    def distance(r1, r2):
        x1, y1, w1, h1 = r1
        x2, y2, w2, h2 = r2
        return max(abs(x1 - x2), abs(y1 - y2))

    # Function to merge a list of rectangles into one
    def merge(rects):
        x_min = min(r[0] for r in rects)
        y_min = min(r[1] for r in rects)
        x_max = max(r[0] + r[2] for r in rects)
        y_max = max(r[1] + r[3] for r in rects)
        return (x_min, y_min, x_max - x_min, y_max - y_min)

    merged = True
    while merged:
        merged = False
        for i in range(len(bboxes)):
            for j in range(i + 1, len(bboxes)):
                if distance(bboxes[i], bboxes[j]) < threshold:
                    bboxes[i] = merge([bboxes[i], bboxes[j]])
                    del bboxes[j]
                    merged = True
                    break
            if merged:
                break

    return bboxes


# Function to check if a rectangle is inside another
def is_inside(r1, r2):
    x1, y1, w1, h1 = r1
    x2, y2, w2, h2 = r2
    return x1 > x2 and y1 > y2 and x1 + w1 < x2 + w2 and y1 + h1 < y2 + h2

# Function to calculate area of a rectangle
def area(bbox):
    _, _, w, h = bbox
    return w * h

def cv2_to_fitz(bbox):
    x, y, w, h = bbox
    return fitz.Rect(x, y, x + w, y + h)


# Open the document
doc = fitz.open(pdfFile)

# Select a page
page = doc[8]

# Remove all annotations from the page
page = remove_annotations(page, extractictionType='both')

# Get the dimensions of the page
dimensions = page.rect
clip_areas = [dimensions]#[fitz.Rect(605, 515, 805, 565),fitz.Rect(815, 515, 1030, 565)]

for clip_area in clip_areas:
    # Get the pixmap
    pix = page.get_pixmap(clip=clip_area)

    # Convert the pixmap to a numpy array
    img = np.frombuffer(pix.samples, dtype=np.uint8).reshape(pix.h, pix.w, pix.n)

    # Convert RGB to BGR
    img_bgr = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)


    # Convert the image to grayscale
    gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)

    # Apply the thresholding  
    thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)

    # Find contours
    contours, _ = cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)#(thresh, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)

    # Get the bounding boxes
    bboxes = [cv2.boundingRect(contour) for contour in contours]

    # Remove rectangles that contain all others
    bboxes = [r for r in bboxes if any(is_inside(r, r2) for r2 in bboxes if r != r2)]

    # Remove rectangles that are contained in others
    bboxes = [r for r in bboxes if not any(is_inside(r, r2) for r2 in bboxes if r != r2)]

    # Use the function on your bounding boxes
    bboxes = merge_close_rectangles(bboxes, 20)  # Adjust the threshold as needed

    # Filter out rectangles with area < 90
    bboxes = [bbox for bbox in bboxes if area(bbox) >= 90]

    # Filter out rectangles with area > 850
    bboxes = [bbox for bbox in bboxes if area(bbox) <= 850]

    # Convert the bounding boxes to fitz.Rect
    image_fitz_bboxes = [cv2_to_fitz(bbox) for bbox in bboxes]

    # Draw the bounding boxes on the image for visualization
    for bbox in bboxes:
        x, y, w, h = bbox
        cv2.rectangle(img_bgr, (x, y), (x+w, y+h), (0, 255, 0), 2)
        # area = w*h
        # print(area)

    # close the document
    doc.close()
    
    # Display the image with bounding boxes
    cv2.imshow('Image with Bounding Boxes', img_bgr)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# clear all memory but 'image_fitz_bboxes'
del globals()['doc']
del globals()['page']
del globals()['dimensions']
del globals()['clip_areas']
del globals()['pix']
del globals()['img']
del globals()['img_bgr']
del globals()['gray']
del globals()['thresh']
del globals()['contours']
del globals()['bboxes']
del globals()['bbox']
del globals()['x']
del globals()['y']
del globals()['w']
del globals()['h']
del globals()['clip_area']

# Run the garbage collector
import gc
gc.collect()


## get the <b>bboxes of the image labels</b>

In [ ]:
import fitz
from PIL import Image,ImageDraw
import io
from IPython import display

# Open the document
doc = fitz.open(pdfFile)

# Select a page
page = doc[8]

# 1st -------------------------------------------------------
# Remove necessary text to get the bboxes of the image labels
# ------------------------------------------------------------
# Get the list of blocks
""" For the display, since pymupdf draw_rect is not erasable we use a pil image of the page """
# render page to an image
pix0 = page.get_pixmap()

# Get the image data
img_data = pix0.tobytes("png")

# Create a PIL image from the data
img0 = Image.open(io.BytesIO(img_data))

# create a drawing context for PIL
draw = ImageDraw.Draw(img0)

for block in page.get_text('dict')['blocks']:
    if 'lines' in block:
        link_text = ''
        for line in block['lines']:
            for span in line['spans']:
                if span['size'] >7.77: # the labels we want to keep have size lower than 7.77
                    searchText = span['text']

                    # We need to add this condition to prevent a code error in the next block
                    if searchText.lower() == 'solution':
                        continue

                    if block['number'] < 5:
                        # Search for the text
                        text_instances = page.search_for(searchText)

                        # Redact the text
                        for inst in text_instances:
                            page.add_redact_annot(inst)
                        page.apply_redactions()
                        # Draw a rectangle on the image
                        # The arguments are two points of the diagonal of the rectangle
                        draw.rectangle(block['bbox'], outline="red")
                        # page.draw_rect(block['bbox'], color=(1, 0, 0), width=0.3)
                else:
                    draw.rectangle(block['bbox'], outline="green")
                    # page.draw_rect(block['bbox'], color=(0, 1, 0), width=0.7)#, fill=(1, 0, 0), width=0.3, overlay=True)

# Display the image - PIL representation of the page
display.display(img0)

# Display the image - The resluts of the redaction in the page object
# Render the page to an image
pix = page.get_pixmap()
# Get the image data from the pixmap
data = pix.samples
# Create a PIL image from the data
img = Image.frombytes("RGB", [pix.width, pix.height], data)
# Display the image
display.display(img)
""" # Save the redacted PDF
doc.save('redacted.pdf', garbage=4, deflate=True, clean=True) """




# 2nd -------------------------------------------------------
# Get the bboxes of the image labels
# ------------------------------------------------------------

textBboxes = []#image_fitz_bboxes
for block in page.get_text('dict')['blocks']:
    if block['number'] >1:
        if block['number'] == 2:
            cult = (block['lines'][0]['spans'][0]['bbox'][0], block['lines'][0]['spans'][0]['bbox'][1], block['lines'][1]['spans'][0]['bbox'][2], block['lines'][1]['spans'][0]['bbox'][3])
            textBboxes.append(cult)
            access = block['lines'][2]['spans'][0]['bbox']
            textBboxes.append(access)
        else:
            textBboxes.append((block['bbox']))
print('---')
print(textBboxes)

# Draw a rectangle on the page for each bbox
for bbox in textBboxes:
    rect = fitz.Rect(*bbox)
    text = page.get_textbox(rect)
    text_single_line = text.replace('\n', '')
    # print(text_single_line)
    page.draw_rect(bbox, color=(1, 0, 0))#, fill=(1, 0, 0), width=0.3, overlay=True)

# Render the page to an image
pix = page.get_pixmap()
# Get the image data from the pixmap
data = pix.samples
# Create a PIL image from the data
img = Image.frombytes("RGB", [pix.width, pix.height], data)
# Display the image
display.display(img)

# close the document
doc.close()


# clear all memory but 'textBboxes'
del globals()['doc']
del globals()['page']
del globals()['pix0']
del globals()['img_data']
del globals()['img0']
del globals()['draw']
del globals()['block']
del globals()['text_instances']
del globals()['inst']
del globals()['pix']
del globals()['data']
del globals()['img']
del globals()['rect']
del globals()['text']
del globals()['text_single_line']
del globals()['bbox']
del globals()['cult']
del globals()['access']
del globals()['span']
del globals()['line']
del globals()['searchText']
del globals()['link_text']


# Run the garbage collector
import gc
gc.collect()


## extract and <b>label the images</b>

In [ ]:
label_dict = {"Cultural and Urban":"Cultural", "Accessibility":"Accessibility", "Semantic Analysis / Active Listening":"Semantic", "Certification":"Certification", 
              "Chatbot /Natural Language Processing":"Chatbot", "Technological Consultancy":"Technological", "Tourism Consultancy":"Tourism_Consultancy", 
              "Tourism CRM":"Tourism_CRM", "App Development":"App", "Graphic Design /Infographics":"Graphic", "Website Design and Development":"Website", 
              "Drones":"Drones", "Visual / Virtual Environments":"Visual", "Efficient Management: Water":"Water", "Efficient Management: Air quality":"Air_quality", 
              "Efficient Management: Energy":"Energy", "Efficient Management: Waste":"Waste", "Artificial Intelligence":"Artificial", "IoT (Internet Things) / Wearables":"IoT", 
              "Marketing /Promotion /Sales":"Marketing", "Mobility":"Mobility", "Other Hardware / Software Solutions":"Other", 
              "Communication Platforms /Training /Webinars":"Communication", "Smart Destination /Smart City Platforms":"Smart", "Health":"Health", "Safety":"Safety", 
              "Sensorisation":"Sensorisation", "Data Analytics Systems (Big data, BI, etc.)":"Data", "Geographic Information Systems (GIS)":"Geographic", 
              "Wi-Fi / Mi-Fi(mobile Wi-Fi)":"Wi-Fi", "Niche":"Niche", "Beach":"Beach", "Nature and Sport":"Nature"}

In [ ]:
def get_closest_left_bbox(text_bboxes, img_bboxes):
    closest_left_bboxes = []
    for text_bbox in text_bboxes:
        min_distance = float('inf')
        closest_left_bbox = None
        for img_bbox in img_bboxes:
            if img_bbox[0] < text_bbox[0]:  # Check if the image bbox is to the left of the text bbox
                # Calculate the distance between the centers of the two bboxes
                distance = ((img_bbox[0] + img_bbox[2]/2 - (text_bbox[0] + text_bbox[2]/2))**2 +
                            (img_bbox[1] + img_bbox[3]/2 - (text_bbox[1] + text_bbox[3]/2))**2)**0.5
                if distance < min_distance:
                    min_distance = distance
                    closest_left_bbox = img_bbox
        closest_left_bboxes.append((text_bbox, closest_left_bbox))
    return closest_left_bboxes

""" !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! """
# Check if the text and image bboxes are the same lenght
# should be the same lenght, if not please rerun the code
print(len(textBboxes)) # --> cell 32
print(len(image_fitz_bboxes)) #  --> cell 29
""" !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! """

# Order the image_fitz_bboxes to the order of the textBboxes list
images = get_closest_left_bbox(textBboxes, image_fitz_bboxes)
# print(image_fitz_bboxes)
print(images)


# Open the document
doc = fitz.open(pdfFile)

# Select a page
page = doc[8]
# extract the raster and vector images from the bboxes and save them with the value from the label dictionary (each key is the text of the bbox)
# we are generating two folders with different labeled images sizes and two extraction forms
for i, bbox in enumerate(images):
    # # 1st -------------------------------------------------------
    # # Convert the text bbox to a fitz.Rect object
    rect = fitz.Rect(*bbox[0])

    # Get the text from the fitz.Rect object
    text = page.get_textbox(rect)
    # Replace the new line characters with a space
    text_single_line = text.replace('\n', '')

    # Convert the image bbox to a fitz.Rect object
    image_rect = fitz.Rect(*bbox[1])#fitz.Rect(*image)

    # get the image (pixmap) from the image_fitz_bboxes
    pix = page.get_pixmap(clip=image_rect)

    # Save the image
    pix.save(f'1stSpanish/raster_from_PDF/small/{label_dict[text_single_line]}.png')


    # Convert the pixmap to a PIL image
    img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)

    # Resize the image
    new_size = (pix.width*10, pix.height*10)  # New size (width, height)
    img_resized = img.resize(new_size)

    # Save the resized image
    img_resized.save(f'1stSpanish/raster_from_PDF/large/{label_dict[text_single_line]}.png')

for i, bbox in enumerate(images):
    # # 2nd -------------------------------------------------------
    # # 1st -------------------------------------------------------
    # # Convert the text bbox to a fitz.Rect object
    rect = fitz.Rect(*bbox[0])

    # Get the text from the fitz.Rect object
    text = page.get_textbox(rect)
    
    # Replace the new line characters with a space
    text_single_line = text.replace('\n', '')

    # Convert the image bbox to a fitz.Rect object
    image_rect = fitz.Rect(*bbox[1])#fitz.Rect(*image)

    # get the image (pixmap) from the image_fitz_bboxes
    pix = page.get_pixmap(clip=image_rect)

    # # save the document to a temporary memory file
    # # Get the byte stream of the PDF
    pdf_bytes = doc.write()

    # Save the byte stream to a BytesIO object
    temp_pdf = io.BytesIO(pdf_bytes)

    # Open the PDF in PyMuPDF
    doc1 = fitz.open(stream=temp_pdf.getvalue(), filetype='pdf')

    # Select a page
    page1 = doc1[8]

    # Set the crop box of the page to your area
    page1.set_cropbox(image_rect)

    # extract the page form the pdf as image
    pix1 = page.get_pixmap()

    # Convert the pixmap to a PIL image
    img1 = Image.frombytes("RGB", [pix1.width, pix1.height], pix1.samples)

    # Save the image
    pix1.save(f'1stSpanish/raster_in_PDF/small/{label_dict[text_single_line]}.png')

    # Resize the image
    new1_size = (pix.width*10, pix.height*10)  # New size (width, height)
    img1_resized = img.resize(new_size)

    # augment(img1_resized, labels=label_dict[text_single_line], amplify=2)

    # Save the resized image
    img1_resized.save(f'1stSpanish/raster_in_PDF/large/{label_dict[text_single_line]}.png')


# Close the document
doc1.close()
# Close the document
doc.close()


#  Clear all memory but images
del globals()['doc']
del globals()['page']
del globals()['rect']
del globals()['text']
del globals()['text_single_line']
del globals()['image_rect']
del globals()['pix']
del globals()['img']
del globals()['new_size']
del globals()['img_resized']
del globals()['pdf_bytes']
del globals()['temp_pdf']
del globals()['doc1']
del globals()['page1']
del globals()['pix1']
del globals()['img1']
del globals()['new1_size']
del globals()['img1_resized']
del globals()['bbox']
# del globals()['images']
del globals()['i']

# Run the garbage collector
import gc
gc.collect()

### functions to process the svg and <b>get (or save) the svg labelled images labbeled image paths an bboxes</b>

In [ ]:

import fitz
import xml.dom.minidom
import numpy as np
import math
from svgpathtools import Path, Line, QuadraticBezier, CubicBezier, Arc, svg2paths, wsvg
from svgpathtools.parser import parse_path
from PIL import Image
import IPython.display as display


class MyDOM(xml.dom.minidom.Document):
    def getElementById(self, element_id):
        return self._get_element_by_id_helper(self, element_id)

    def _get_element_by_id_helper(self, node, element_id):
        for child in node.childNodes:
            if child.nodeType == xml.dom.Node.ELEMENT_NODE:
                if child.getAttribute('id') == element_id:
                    return child
                result = self._get_element_by_id_helper(child, element_id)
                if result is not None:
                    return result
        return None

    def get_parent_map(self):
        parent_map = {c: p for p in self.iter() for c in p}
        return parent_map


# import xml.etree.ElementTree as ET

# def set_fill_color(filename, color):
#     # Parse the SVG file
#     tree = ET.parse(filename)
#     root = tree.getroot()

#     # Define the SVG namespace
#     namespaces = {'svg': 'http://www.w3.org/2000/svg'}

#     # Find all path elements
#     for path in root.findall('svg:path', namespaces):
#         # Set the fill color
#         path.set('fill', color)

#     # Write the modified XML back to the file
#     tree.write(filename)


    
# ----------------------------------------------------------------------------------------------------------------------------------
# 00000000000000000000000000000000000000 <START TRANSFORMATION FUNCTION FAMILY> 0000000000000000000000000000000000000000000000000000
# ----------------------------------------------------------------------------------------------------------------------------------   
def skew_path(path_data, skew_angle, axis='x'):
    # Convert the skew angle to radians
    skew_angle_rad = math.radians(skew_angle)    

    # Define the skew transformation matrix
    if axis == 'x':
        skew_matrix = np.array([[1, math.tan(skew_angle_rad), 0], [0, 1, 0]])
    elif axis == 'y':
        skew_matrix = np.array([[1, 0, 0], [math.tan(skew_angle_rad), 1, 0]])

    # Apply the skew transformation to each point in the path
    skewed_path = []
    
    print('-----------------\nskew_data\n-----------------')
    for segment in path_data:
        print('skew_segment from:',str(segment)[:100])
        if isinstance(segment, Line):
            # Handle 'Line' command
            point1 = segment.start
            point2 = segment.end
            skewed_point1 = np.dot(skew_matrix, [point1.real, point1.imag, 1])
            skewed_point2 = np.dot(skew_matrix, [point2.real, point2.imag, 1])
            skewed_path.append(Line(start=complex(skewed_point1[0], skewed_point1[1]),end=complex(skewed_point2[0], skewed_point2[1])))
        elif isinstance(segment, CubicBezier):
            # Handle 'CubicBezier' command
            point0 = segment.start
            point1 = segment.control1
            point2 = segment.control2
            point3 = segment.end
            skewed_point0 = np.dot(skew_matrix, [point0.real, point0.imag, 1])
            skewed_point1 = np.dot(skew_matrix, [point1.real, point1.imag, 1])
            skewed_point2 = np.dot(skew_matrix, [point2.real, point2.imag, 1])
            skewed_point3 = np.dot(skew_matrix, [point3.real, point3.imag, 1])
            skewed_path.append((CubicBezier(complex(skewed_point0[0], skewed_point0[1]),complex(skewed_point1[0], skewed_point1[1]), complex(skewed_point2[0], skewed_point2[1]), complex(skewed_point3[0], skewed_point3[1]))))
        elif isinstance(segment, QuadraticBezier):
            # Handle 'QuadraticBezier' command
            point0 = segment.start
            point1 = segment.control
            point2 = segment.end
            skewed_point0 = np.dot(skew_matrix, [point0.real, point0.imag, 1])
            skewed_point1 = np.dot(skew_matrix, [point1.real, point1.imag, 1])
            skewed_point2 = np.dot(skew_matrix, [point2.real, point2.imag, 1])
            skewed_path.append((QuadraticBezier(complex(skewed_point0[0], skewed_point0[1]),complex(skewed_point1[0], skewed_point1[1]), complex(skewed_point2[0], skewed_point2[1]))))
        elif isinstance(segment, Arc):
            # Handle 'Arc' command
            point0 = segment.start
            point1 = segment.end
            skewed_point0 = np.dot(skew_matrix, [point0.real, point0.imag, 1])
            skewed_point1 = np.dot(skew_matrix, [point1.real, point1.imag, 1])
            skewed_path.append((Arc(complex(skewed_point0[0], skewed_point0[1]), complex(skewed_point1[0], skewed_point1[1]), segment.radius, segment.rotation, segment.large_arc, segment.sweep)))     
        else:
            print('Skew - did it come here?')
            # Handle other commands (assuming they are iterable)
            for point in segment:
                skewed_point = np.dot(skew_matrix, [point.real, point.imag, 1])
                skewed_path.append((complex(skewed_point[0], skewed_point[1])))
        print('skew_segment to:',str(skewed_path[-1])[:100])                 

    return skewed_path
# ----------------------------------------------------------------------------------------------------------------------------------  
def scale_path(path_data, scale_x, scale_y):
    print('\n scale_path started', path_data, scale_x, scale_y)
    # Apply the scale transformation to each point in the path
    scaled_path = []

    print('-----------------\nscale_data\n-----------------')
    for segment in path_data:
        print('scale_segment from:',str(segment)[:100])
        if isinstance(segment, Line):
            # Handle 'Line' command
            point1 = segment.start
            point2 = segment.end
            scaled_point1 = complex(point1.real * scale_x, point1.imag * scale_y)
            scaled_point2 = complex(point2.real * scale_x, point2.imag * scale_y)
            scaled_path.append(Line(start=complex(scaled_point1.real, scaled_point1.imag),end=complex(scaled_point2.real, scaled_point2.imag)))
        elif isinstance(segment, CubicBezier):
            # Handle 'CubicBezier' command
            point0 = segment.start
            point1 = segment.control1
            point2 = segment.control2
            point3 = segment.end
            scaled_point0 = complex(point0.real * scale_x, point0.imag * scale_y)
            scaled_point1 = complex(point1.real * scale_x, point1.imag * scale_y)
            scaled_point2 = complex(point2.real * scale_x, point2.imag * scale_y)
            scaled_point3 = complex(point3.real * scale_x, point3.imag * scale_y)
            scaled_path.append((CubicBezier(complex(scaled_point0.real, scaled_point0.imag),complex(scaled_point1.real, scaled_point1.imag), complex(scaled_point2.real, scaled_point2.imag), complex(scaled_point3.real, scaled_point3.imag))))
        elif isinstance(segment, QuadraticBezier):
            # Handle 'QuadraticBezier' command
            point0 = segment.start
            point1 = segment.control
            point2 = segment.end
            scaled_point0 = complex(point0.real * scale_x, point0.imag * scale_y)
            scaled_point1 = complex(point1.real * scale_x, point1.imag * scale_y)
            scaled_point2 = complex(point2.real * scale_x, point2.imag * scale_y)
            scaled_path.append((QuadraticBezier(complex(scaled_point0.real, scaled_point0.imag),complex(scaled_point1.real, scaled_point1.imag), complex(scaled_point2.real, scaled_point2.imag))))
        elif isinstance(segment, Arc):
            # Handle 'Arc' command
            point0 = segment.start
            point1 = segment.end
            scaled_point0 = complex(point0.real * scale_x, point0.imag * scale_y)
            scaled_point1 = complex(point1.real * scale_x, point1.imag * scale_y)
            scaled_path.append((Arc(complex(scaled_point0.real, scaled_point0.imag), complex(scaled_point1.real, scaled_point1.imag), segment.radius, segment.rotation, segment.large_arc, segment.sweep)))        
        else:
            print('Scale - did it come here?')
            # Handle other commands (assuming they are iterable)
            for point in segment:
                scaled_point = complex(point.real * scale_x, point.imag * scale_y)
                scaled_path.append((complex(scaled_point.real, scaled_point.imag)))
        print('scale_segment to:',str(scaled_path[-1])[:100]) 

    return scaled_path
# ----------------------------------------------------------------------------------------------------------------------------------  
def translate_path(path_data, dx, dy):
    print('translate_path started', path_data, dx, dy)
    # Apply the translation transformation to each point in the path
    translated_path = []

    print('-----------------\ntranslate_data\n-----------------')
    for segment in path_data:
        print('translate_segment from:',str(segment)[:100])
        if isinstance(segment, Line):
            # Handle 'Line' command
            point1 = segment.start
            point2 = segment.end
            translated_point1 = complex(point1.real + dx, point1.imag + dy)
            translated_point2 = complex(point2.real + dx, point2.imag + dy)
            translated_path.append(Line(start=complex(translated_point1.real, translated_point1.imag),end=complex(translated_point2.real, translated_point2.imag)))
        elif isinstance(segment, CubicBezier):
            # Handle 'CubicBezier' command
            point0 = segment.start
            point1 = segment.control1
            point2 = segment.control2
            point3 = segment.end
            translated_point0 = complex(point0.real + dx, point0.imag + dy)
            translated_point1 = complex(point1.real + dx, point1.imag + dy)
            translated_point2 = complex(point2.real + dx, point2.imag + dy)
            translated_point3 = complex(point3.real + dx, point3.imag + dy)
            translated_path.append((CubicBezier(complex(translated_point0.real, translated_point0.imag),complex(translated_point1.real, translated_point1.imag), complex(translated_point2.real, translated_point2.imag), complex(translated_point3.real, translated_point3.imag))))
        elif isinstance(segment, QuadraticBezier):
            # Handle 'QuadraticBezier' command
            point0 = segment.start
            point1 = segment.control
            point2 = segment.end
            translated_point0 = complex(point0.real + dx, point0.imag + dy)
            translated_point1 = complex(point1.real + dx, point1.imag + dy)
            translated_point2 = complex(point2.real + dx, point2.imag + dy)
            translated_path.append((QuadraticBezier(complex(translated_point0.real, translated_point0.imag),complex(translated_point1.real, translated_point1.imag), complex(translated_point2.real, translated_point2.imag))))
        elif isinstance(segment, Arc):
            # Handle 'Arc' command
            point0 = segment.start
            point1 = segment.end
            translated_point0 = complex(point0.real + dx, point0.imag + dy)
            translated_point1 = complex(point1.real + dx, point1.imag + dy)
            translated_path.append((Arc(complex(translated_point0.real, translated_point0.imag), complex(translated_point1.real, translated_point1.imag), segment.radius, segment.rotation, segment.large_arc, segment.sweep)))        
        else:
            print('Translate - did it come here?')
            # Handle other commands (assuming they are iterable)
            for point in segment:
                translated_point = complex(point.real + dx, point.imag + dy)
                translated_path.append((complex(translated_point.real, translated_point.imag)))
        print('translate_segment to:',str(translated_path[-1])[:100])


    return translated_path
# ----------------------------------------------------------------------------------------------------------------------------------
# 0000000000000000000000000000000000000000000000000000 <END FUNCTION> 00000000000000000000000000000000000000000000000000000000000000
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~	


# ----------------------------------------------------------------------------------------------------------------------------------
# 0000000000000000000000000000000000000000000000000000 <START FUNCTION> 000000000000000000000000000000000000000000000000000000000000
# ----------------------------------------------------------------------------------------------------------------------------------
def transform_path(path, transform_matrix):
    # it requires path element as inptut, not path_data as input, so it is necessary to parse the path_data to path element before calling the function transform_path2
    # parse the path data
    path_data = parse_path(path.getAttribute('d')) 

    # Apply the transformation operations to the path. The order of operations should be: skewing, scaling, and then translation
    # Parse the transformation matrix from the string
    a, b, c, d, e, f = map(float, transform_matrix.strip('matrix()').split(','))
    # Apply the transformation matrix to the path
    # ------Skewing------
    """ The skewing parameters b and c in the transformation matrix are not angles, but the tangent of the skew angles.
    Calculating the skew angle from the tangent in degrees """
    skew_angle_x = math.degrees(math.atan(b))
    skew_angle_y = math.degrees(math.atan(c))
    if skew_angle_x != 0:
        print('1st skew_angle_x', skew_angle_x)
        path_object = skew_path(path_data, skew_angle_x, axis='x')
    else:
        path_object = path_data
    if skew_angle_y != 0:
        print('2ndskew_angle_y', skew_angle_y)
        path_object = skew_path(path_object, skew_angle_y, axis='y')
    # ------Scaling------
    if a != 1 or d != 1:
        print('scaling (x, y)', a, d)
        path_object = scale_path(path_object, a, d)
    # ------Translation------
    if e != 0 or f != 0:
        print('Translating (x, y)', e, f)
        path_object = translate_path(path_object, e, f)

    # -----------------------------------------------------------------------------------------------------------------------
    # Convert the path data back to a Path object string. path element is a list of segments, each segment is a list of points, so we cannot retrieve d() the path element as it is, we need to convert it to a Path object
    path_element = Path(*path_object)

    # Update the path data in the SVG document, it we update the path element, it will be updated in the svg document to move to() and wsvg cannot use it
    path.setAttribute('d', path_element.d())

    return path_element
# ----------------------------------------------------------------------------------------------------------------------------------
# 0000000000000000000000000000000000000000000000000000 <END FUNCTION> 00000000000000000000000000000000000000000000000000000000000000
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~	


# ----------------------------------------------------------------------------------------------------------------------------------
# 0000000000000000000000000000000000000000000000000000 <START FUNCTION> 000000000000000000000000000000000000000000000000000000000000
# ----------------------------------------------------------------------------------------------------------------------------------
def get_bbox(path):
    min_x = np.inf
    min_y = np.inf
    max_x = -np.inf
    max_y = -np.inf

    for segment in path:
        x0, x1 = sorted([segment.start.real, segment.end.real])
        y0, y1 = sorted([segment.start.imag, segment.end.imag])

        min_x = min(min_x, x0)
        min_y = min(min_y, y0)
        max_x = max(max_x, x1)
        max_y = max(max_y, y1)
 
    return min_x, max_x, min_y, max_y
# ----------------------------------------------------------------------------------------------------------------------------------
# 0000000000000000000000000000000000000000000000000000 <END FUNCTION> 00000000000000000000000000000000000000000000000000000000000000
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~



# ----------------------------------------------------------------------------------------------------------------------------------
# 0000000000000000000000000000000000000000000000000000 <START FUNCTION> 000000000000000000000000000000000000000000000000000000000000
# ----------------------------------------------------------------------------------------------------------------------------------
def process_element(element, target, pg_nr, elements_list, gloss_bboxes, page_elements_dict, container_bbox,bboxes_rect_fitz, bboxes_rect_svg, page1):
    """ target is the intended application of the function. can be:
     glossary - for the image glossary of the catalogue
        pg - for the soltions in each page of the catalogue """
    
    path_element = None
    transform_matrix = None
    # print('process_element started')
    print(f'this is the element: {element} of type: {type(element)}')
    
    if element.nodeType == element.ELEMENT_NODE:
        if element.tagName == 'g':
            for child in element.childNodes:
                if child is not None and child.nodeType == child.ELEMENT_NODE:
                    process_element(child, target, pg_nr, elements_list, gloss_bboxes, page_elements_dict, container_bbox,bboxes_rect_fitz, bboxes_rect_svg, page1)
        # elif element.nodeType == xml.dom.Node.ELEMENT_NODE and element.tagName in ['path', 'use']:
        if element.tagName == 'use':
            print("in element.tagName == 'use':")
            # This is a 'use' element, so we need to find the corresponding 'path' element
            path_id = element.getAttribute('xlink:href')[1:]  # Remove the '#' at the start
            path_element = my_dom.getElementById(path_id)
            transform_matrix = element.getAttribute('transform') if element.hasAttribute('transform') else "matrix(1,0,0,1,0,0)"
            if path_element is None:
                print(f"Warning: No path element found with ID {path_id}")
                return
            else:
                print('got it!', path_id)
        elif element.tagName == 'path':
            print("in element.tagName == 'path':")#,len(element.childNodes), element.childNodes[0], type(element.childNodes[0]))
            # This is a 'path' element, so we can use it directly
            path_element = element
            transform_matrix = element.getAttribute('transform') if element.hasAttribute('transform') else "matrix(1,0,0,1,0,0)"
            print(path_element.getAttribute('d'), transform_matrix)

        else:
            print(f"Warning: Unhandled element type {element.tagName}")
            return

    else:
        print('in else:')
        # This is not a 'path' or 'use' element, so we can ignore it
        return
    
    print('what is this element thas generating the error?', path_element, type(path_element))#, path_element.getAttribute('d'), transform_matrix, type(transform_matrix
    print(f'\ntransform matrix for: {str(path_element)[:25]} --> {transform_matrix}')
    
    # Apply the transformation operations to the path.
    transformed_path = transform_path(path_element, transform_matrix)
    
    # -----------------------------------------------------------------------------------------------------------------------
    # # Get the bounding box of the tranformed path
    # xmin, xmax, ymin, ymax = transformed_path.bbox()
    # Create a rectangle path for the bounding box
    # bbox_path = parse_path(f'M {xmin} {ymin} L {xmax} {ymin} L {xmax} {ymax} L {xmin} {ymax} Z')
    
    got_bbox = get_bbox(transformed_path)#min_x, max_x, min_y, max_y
    
    print('returned BBOX:',got_bbox,'\n','returned BBOX elements:', got_bbox[0], got_bbox[1], got_bbox[2], got_bbox[3], '\n', 'returned BBOX type:', type(got_bbox[0]), type(got_bbox[1]), type(got_bbox[2]), type(got_bbox[3]))

    # Create a rectangle path for the bounding box
    if got_bbox[0] != np.inf and got_bbox[1] != -np.inf and got_bbox[2] != np.inf and got_bbox[3] != -np.inf:

        # Create a rectangle path for the bounding box
        rect_svg = parse_path(f'M {got_bbox[0]} {got_bbox[2]} L {got_bbox[1]} {got_bbox[2]} L {got_bbox[1]} {got_bbox[3]} L {got_bbox[0]} {got_bbox[3]} Z')

        # Add the bounding svg box to the list
        bboxes_rect_svg.append(rect_svg)

        # create a fitz rectangle
        rect_fitz = fitz.Rect(got_bbox[0], got_bbox[2], got_bbox[1], got_bbox[3])#got_bbox[0], svg_height - got_bbox[2], got_bbox[1], svg_height - got_bbox[3])

        # add the fitz rectangle to the list
        bboxes_rect_fitz.append(rect_fitz)


        # -----------------------------------------------------------------------------------------------------------------------    
        if target == 'glossary':
            # ---- check if path is in 'images' and append to elements_list {'label': [element]}

            for image in images:
                # # Convert the text bbox to a fitz.Rect object
                rect = fitz.Rect(image[0])
                
                # Get the text from the fitz.Rect object
                text = page1.get_textbox(rect)
                # Replace the new line characters with a space
                text_single_line = text.replace('\n', '')
                # print(label_dict[text_single_line])   
                if got_bbox[0] >= image[1][0] and got_bbox[2] >= image[1][1] and got_bbox[1] <= image[1][2] and got_bbox[3] <= image[1][3]:
                    # append the svg path to a list in a dictionary in the 'elements_list', with the label text as key

                    # Get the label from the label dictionary
                    label = label_dict[text_single_line]

                    # Find if this label dictionary already exists in elements_list (Dict that has 'label' as a key)
                    dict_with_label = next((d for d in elements_list if label in d), None)
                    dict_with_gloss_bboxes = next((d for d in gloss_bboxes if label in d), None)

                    # append the element to the list of the dictionary with the label as key
                    if dict_with_label:
                        # If the dictionary exists, append 'element' to the list
                        dict_with_label[label].append(transformed_path)
                    else:
                        # If the dictionary doesn't exist, create a new one
                        elements_list.append({label: [transformed_path]})# this list of dicts stores the svg paths from the glossary
                        # dict_with_label[label] = [transformed_path]

                    if dict_with_gloss_bboxes:
                        # If the dictionary exists, append bbox 'element' to the list 
                        dict_with_gloss_bboxes[label].append([got_bbox[1],got_bbox[3],got_bbox[0],got_bbox[2]])
                    else:
                        # If th dictionary doesn't exist, create a new one
                        gloss_bboxes.append({label: [[got_bbox[1],got_bbox[3],got_bbox[0],got_bbox[2]]]})# this list of dicts stores the bboxes of the svg paths from the glossary, to create a viewbox for the svg

            # # -----------------------------------------------------------------------------------------------------------------------
            # # Write the bounding box paths to a new SVG file
            # wsvg(bboxes_rect_svg, filename='output.svg')
            

        elif target == 'pg':
            print('(is it getting to traget page?)')
            """ check if path in destType bbox [600, 515, 810, 565], or solType bbox [810, 515, 1035, 565] """
            """ and apend to elements_list {'label': [element]} """
            label = None
            if got_bbox[0] >= 600 and got_bbox[2] >= 515 and got_bbox[1] <= 810 and got_bbox[3] <= 565:
                # Set the label 
                label = 'destType'
            elif got_bbox[0] >= 810 and got_bbox[2] >= 515 and got_bbox[1] <= 1035 and got_bbox[3] <= 565:
                # Set the label 
                label = 'solType'
            else:
                print(f'bbox is not in destType or solType in page {pg_nr}: {got_bbox}')
                return
            if label:
                
                # check if it is a container bbox and append to container_bbox {pg_nr: {label: [bbox]}}
                if (got_bbox[1]-got_bbox[0])*(got_bbox[3]-got_bbox[2]) > 1000:
                    # global container_bbox
                    if pg_nr not in container_bbox:
                        container_bbox[pg_nr] = {label: [got_bbox]}

                    elif pg_nr in container_bbox:
                        if label in container_bbox[pg_nr]:
                            container_bbox[pg_nr][label].append(got_bbox)
                        else:
                            container_bbox[pg_nr][label] = [got_bbox]
                            
                
                elif (got_bbox[1]-got_bbox[0])*(got_bbox[3]-got_bbox[2]) < 1000:# (x_max-x_min)*(y_max-y_min)

                    if pg_nr not in page_elements_dict:
                        page_elements_dict[pg_nr] = {label: [transformed_path]}
                    elif pg_nr in page_elements_dict:
                        if label in page_elements_dict[pg_nr]:
                            page_elements_dict[pg_nr][label].append(transformed_path)# this dict stores the svg paths from the looped pages
                        else:
                            page_elements_dict[pg_nr][label] = [transformed_path]
                    # -----------------------------------------------------------------------------------------------------------------------                    
                    global counter_proc
                    counter_proc += 1
                    # wsvg(transformed_path, filename=f'1stSpanish/test/pag_{pg_nr}_out_{label}_{counter_proc}.svg')
                   
                else:
                    print(f'bbox is not in destType or solType in page {pg_nr}: {got_bbox}')
                    return
            

    # Recursively process child elements
    for child in element.childNodes:
        print('in for child in element.childNodes:')
        target = target
        pg_nr = pg_nr
        elements_list = elements_list
        gloss_bboxes = gloss_bboxes
        page_elements_dict = page_elements_dict
        container_bbox = container_bbox
        bboxes_rect_fitz = bboxes_rect_fitz
        bboxes_rect_svg = bboxes_rect_svg
        page1 = page1
        process_element(child, target, pg_nr, elements_list, gloss_bboxes, page_elements_dict, container_bbox,bboxes_rect_fitz, bboxes_rect_svg, page1)
        


# ----------------------------------------------------------------------------------------------------------------------------------
# 0000000000000000000000000000000000000000000000000000 <END FUNCTION> 00000000000000000000000000000000000000000000000000000000000000
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


# ----------------------------------------------------------------------------------------------------------------------------------
# 0000000000000000000000000000000000000000000000000000 <START FUNCTION> 000000000000000000000000000000000000000000000000000000000000
# ----------------------------------------------------------------------------------------------------------------------------------
# translate the path to origin (0,0)
def translate_path_to_origin(path_string):

    path_in_origin = []
    """Parse an SVG path string into a list of path segments."""
    # path_data = parse_path(path_string)

    # Calculate the bounding box
    min_x, max_x, min_y, max_y = get_bbox(path_string)
    print(min_x, max_x, min_y, max_y)
    
    # Translate the path
    # path_in_origin = translate_path(path_data, - min_x, - min_y)
    for point in path_string:
        if isinstance(point, CubicBezier):
            start = complex(point.start.real - min_x, point.start.imag - min_y)
            control1 = complex(point.control1.real - min_x, point.control1.imag - min_y)
            control2 = complex(point.control2.real - min_x, point.control2.imag - min_y)
            end = complex(point.end.real - min_x, point.end.imag - min_y)
            path_in_origin.append(CubicBezier(start, control1, control2, end))
        elif isinstance(point, QuadraticBezier):
            start = complex(point.start.real - min_x, point.start.imag - min_y)
            control = complex(point.control.real - min_x, point.control.imag - min_y)
            end = complex(point.end.real - min_x, point.end.imag - min_y)
            path_in_origin.append(QuadraticBezier(start, control, end))
        elif isinstance(point, Arc):
            start = complex(point.start.real - min_x, point.start.imag - min_y)
            end = complex(point.end.real - min_x, point.end.imag - min_y)
            path_in_origin.append(Arc(start, end, point.radius, point.rotation, point.large_arc, point.sweep))
        elif isinstance(point, Line):
            start = complex(point.start.real - min_x, point.start.imag - min_y)
            end = complex(point.end.real - min_x, point.end.imag - min_y)
            path_in_origin.append(Line(start, end))

        # -----------------------------------------------------------------------------------------------------------------------
    # Convert the path data back to a Path object string. path element is a list of segments, each segment is a list of points, so we cannot retrieve d() the path element as it is, we need to convert it to a Path object
    path_element = Path(*path_in_origin)

    # # Update the path data in the SVG document, it we update the path element, it will be updated in the svg document to move to() and wsvg cannot use it
    # path.setAttribute('d', path_element.d())

    # calculate the new bbox
    min_x, max_x, min_y, max_y = get_bbox(path_element)


    return path_element, min_x, max_x, min_y, max_y
# ----------------------------------------------------------------------------------------------------------------------------------
# 0000000000000000000000000000000000000000000000000000 <END FUNCTION> 00000000000000000000000000000000000000000000000000000000000000
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


### function (and its call) to preview the <b>bboxes of the 'image glossary' svg image</b>,  and <b>get (or save) the svg glossary labelled images</b>

In [ ]:
# ||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
# 00000000000000000000000000000000000000000000000000 'self' code block 0000000000000000000000000000000000000000000000000000000000000


import fitz
from PIL import Image
import IPython.display as display
import sys
class NullWriter(object):
    def write(self, arg):
        pass

def process_pdf_gloss_svg(doc,page_nr):
    # global bboxes_rect_svg
    # global bboxes_rect_fitz
    # global elements_list
    # global label_dict
    
    # global gloss_bboxes
    # gloss_bboxes = []
    global counter_proc
    counter_proc = 0

    elements_list = []
    gloss_bboxes = []
    page_elements_dict = {}
    container_bbox = {}
    bboxes_rect_fitz = []
    bboxes_rect_svg = []

    # Select a page for transformation
    page = doc[page_nr]

    # Get a page for text extraction
    # # save the document to a temporary memory file
    # # Get the byte stream of the PDF
    pdf_bytes = doc.write()

    # Save the byte stream to a BytesIO object
    temp_pdf = io.BytesIO(pdf_bytes)

    # Open the PDF in PyMuPDF
    doc1 = fitz.open(stream=temp_pdf.getvalue(), filetype='pdf')

    # Select a page
    page1 = doc1[page_nr]

    # Remove annotations from the page
    page = remove_annotations(page,'both') # It works without removing anotations, but it gets very slow -- > this function is in cell 26

    # Save svg image of the page
    svg_string = page.get_svg_image()

    # Parse the SVG
    svg_doc = xml.dom.minidom.parseString(svg_string)
    svg_element = svg_doc.documentElement

    # Create an instance of MyDOM and import the SVG document into it
    global my_dom
    my_dom = MyDOM()
    my_dom.appendChild(my_dom.importNode(svg_element, True))

    # Get the svg canvas height. (height is string, so we need to convert it to float, but it has 'pt' -> <class 'str'> 1497.48pt)
    svg_height = float(my_dom.documentElement.getAttribute('height')[:-2])
    results_path = []
    results_bbox = []
    # Process all child elements of the SVG element
    for child in my_dom.documentElement.childNodes:
        # print('this is the child type:', type(child))

        # --------------
        # do not print this section -> svg_string = path.d()
        nullwrite = NullWriter()
        oldstdout = sys.stdout
        sys.stdout = nullwrite # disable output        
        # Scale path
        """ for 'page' we must pass: page_elements_dict, container_bbox
        for 'glossary' we must pass: elements_list, gloss_bboxes """
        process_element(child,'glossary', page_nr, elements_list, gloss_bboxes, page_elements_dict, container_bbox,bboxes_rect_fitz, bboxes_rect_svg,page1)
        sys.stdout = oldstdout # enable output        
        # --------------

       
        

    # draw the fitz rectangles of the svgs on the page
    for box in bboxes_rect_fitz:
        page.draw_rect(box, color=(0, 1, 0))   

    # Render the page to an image
    pix = page.get_pixmap()

    # Get the image data from the pixmap
    data = pix.samples

    # Create a PIL image from the data
    img = Image.frombytes("RGB", [pix.width, pix.height], data)

    # Display the image
    display.display(img)

    # ?????????????????? --> WORKING <-- ????????????????????????????????????


    # loop through the svg path elements and save the ones in the bbox
    # extract the vector images from the bboxes and save them with the value from the label dictionary (each key is the text of the bbox)
    # uses 'images' and  'label_dict' from cell 36
    # loop through the svg path elements

    
    for entry in elements_list:
        paths = []

        # write a svg file for each label
        for label, path_elements in entry.items():

            # flatten the path_elements list
            flattened = [val for sublist in path_elements for val in sublist]

            # print(f'this is the flattened {label}', flattened)

            wsvg(flattened, filename=f'1stSpanish/glossary_SVG/{label}.svg')

    # ???????????????????????????????????????????????????????????????????????????????????????????
    return elements_list, gloss_bboxes

# doc = fitz.open(pdfFile)
# process_pdf_gloss_svg(doc, 8)

In [ ]:
def process_pdf_page_svg(page, page_nr):
    # global bboxes_rect_svg
    # global bboxes_rect_fitz
    # global elements_list
    # global label_dict
    
    # global gloss_bboxes
    # gloss_bboxes = []
    global counter_proc
    counter_proc = 0

    elements_list = []
    gloss_bboxes = []
    page_elements_dict = {}
    container_bbox = {}
    bboxes_rect_fitz = []
    bboxes_rect_svg = []

    page1 = None

    # Remove annotations from the page
    page = remove_annotations(page,'both') # It works without removing anotations, but it gets very slow -- > this function is in cell 26

    # Save svg image of the page
    svg_string = page.get_svg_image()

    # Parse the SVG
    svg_doc = xml.dom.minidom.parseString(svg_string)
    svg_element = svg_doc.documentElement

    # Create an instance of MyDOM and import the SVG document into it
    global my_dom
    my_dom = MyDOM()
    my_dom.appendChild(my_dom.importNode(svg_element, True))

    # Process all child elements of the SVG element
    for child in my_dom.documentElement.childNodes:
        # print('this is the child type:', type(child))
        """ for 'pg' we must pass: page_elements_dict, container_bbox
        for 'glossary' we must pass: elements_list, gloss_bboxes """
        process_element(child,'pg', page_nr, elements_list, gloss_bboxes, page_elements_dict, container_bbox,bboxes_rect_fitz, bboxes_rect_svg,page1)
      
    return page_elements_dict, container_bbox

# import sys
# class NullWriter(object):
#     def write(self, arg):
#         pass




# # Read the PDF file
# doc = fitz.open(pdfFile)
# results_elements = {}
# results_containers = {}
# # Iterate through the pages
# for i in range(11, 263):
#     page = doc[i]
#     # print(i)
#     # --------------
#     # do not print this section -> svg_string = path.d()
#     nullwrite = NullWriter()
#     oldstdout = sys.stdout
#     sys.stdout = nullwrite # disable output        
#     # Scale path
#     result = process_pdf_page_svg(page, i)
#     sys.stdout = oldstdout # enable output        
#     # --------------
#     results_elements[i] = result[0][i]
#     results_containers[i] = result[1][i]

# # print(results_elements[11]['solType'][0:1])
# # print(results_containers[11]['solType'][0:1])
    

# <span style="color:yellow"><b>identify, transform and compare the paths in the target pages</b></span>

# Loop the pages to analyse

In [ ]:
import gc
import os
import fitz
# Read the PDF file
doc = fitz.open(pdfFile)
print('page','Δ', 'block number','Δ', 'size','Δ', 'flags', 'Δ', 'font','Δ', 'color','Δ', 'ascender', 'descender','Δ', 'text','Δ', 'line_bbox','Δ', 'line','Δ', 'block')

# Iterate through the pages
for i in range(11, 263):#(8, 263):
    
    page = doc[i]

    for block in page.get_text('dict')['blocks']:
        # check if all lines in block have the same color, font and aproximatelly the same size
        if 'lines' in block:
            for line in block['lines']:
                for span in line['spans']:
                    print(i,'Δ', block['number'],'Δ', span['size'],'Δ', span['flags'], 'Δ', span['font'],'Δ', span['color'],'Δ', span['ascender'], span['descender'],'Δ', span['text'],'Δ', line['bbox'],'Δ', line,'Δ', block)

# groups the svg paths, that are inside the containers (the surounding rectangles within each page)

In [ ]:
import json



# # Use json.dumps() to pretty print
# print("\n", json.dumps(container_bbox, indent=4), "\n")
def goup_pg_svgs(page_elements_dict,container_bbox):
    path_groups = {}
    path_groups_view = {}

    for page, val in page_elements_dict.items():# the dictionary where we stored all the path elements in each page falling into the destination and solution type areas (but the rectangles containing the actual elements to analyse)
        
        for labeled, path_elements in val.items():

            for path_element in path_elements:

                x_min_el, x_max_el, y_min_el, y_max_el = get_bbox(path_element)

                if page in container_bbox.keys(): # The dict where we stored the rectangles surrounding of the elements to analyse
                    if labeled in container_bbox[page].keys():
                        for i, coordinates in enumerate(container_bbox[page][labeled]):
                            print('coordinates', coordinates)
                            
                            # get the dimensions of the paths in the svg file
                            x_min, x_max, y_min, y_max = coordinates

                            print(f'got coordinates for {page}_{labeled}_{i}:', x_min, x_max, y_min, y_max)

                            # group the svg paths inside each rectangle container
                            if x_min_el >= x_min and y_min_el >= y_min and x_max_el <= x_max and y_max_el <= y_max:
                                print(f'{page}_{labeled}_{i}', 'is in the bbox of', f'{page}_{labeled}_{i}')

                                key_name = i

                                if page not in path_groups:
                                    # create a new dictionary with the labeled as key and a dict as value, with i as key and a list with the path_element as value, in path_groups
                                    path_groups[page] = {labeled: {i:[path_element]}}
                                    path_groups_view[page] = {labeled: {i:[str(path_element)[0:100]]}}

                                elif page in path_groups:

                                    if labeled not in path_groups[page]:

                                        path_groups[page][labeled] = {i:[path_element]}
                                        path_groups_view[page][labeled] = {i:[str(path_element)[0:100]]}

                                    elif labeled in path_groups[page]:

                                        if i not in path_groups[page][labeled]:

                                            path_groups[page][labeled][i] = [path_element]
                                            path_groups_view[page][labeled][i] = [str(path_element)[0:100]]

                                        elif i in path_groups[page][labeled]:

                                            path_groups[page][labeled][i].append(path_element)
                                            path_groups_view[page][labeled][i].append(str(path_element)[0:100])
    return path_groups, path_groups_view

# Glossary svgs
Creates a dict with the numpy array of the raster from the svg image (np.array(raster))

In [ ]:
# ! pip install cairosvg

from io import BytesIO
from PIL import Image
import cairosvg
import numpy as np
import cv2
from skimage.metrics import structural_similarity as ssim
import sys

# # target size for the images
# pix_target = 500

class NullWriter(object):
    def write(self, arg):
        pass

gloss_height = []
# gloss_np_dict = {}


def process_glossary_svg(elements_list):
    gloss_np_dict = {}
    # target size for the images
    pix_target = 500
    for i, entry in enumerate(elements_list):
        
        for label, path_elements in entry.items():
            # flatten the path_elements list
            path_flattened = [val for sublist in path_elements for val in sublist]
            # print(flattened[:10])

            # Translate path to origin [0] and get the dim [1:]
            path_in_origin = translate_path_to_origin(path_flattened)

            # do not print this section -> svg_string = path.d()
            nullwrite = NullWriter()
            oldstdout = sys.stdout
            sys.stdout = nullwrite # disable output        
            # Scale path
            scale_factor = pix_target/(path_in_origin[2]-path_in_origin[1])
            path_scaled = scale_path(path_in_origin[0], scale_factor, scale_factor)
            sys.stdout = oldstdout # enable output        
            # --------------

            # Convert list of svgpathtools.path.Line objects to a Path object
            path = Path(*path_scaled)

            # Convert path to svg string
            svg_path = path.d()

            # print(gloss_bboxes[i].values())        

            # print(f'<svg xmlns="http://www.w3.org/2000/svg" x="{min_x}" y="{min_y}" width="{max_x - min_x}px" height="{max_y - min_y}px" viewBox="0 0 0 0"><path d="{svg_string}" fill="#000000" stroke="#000000" stroke-width="0.5" /></svg>')

            # if (round(path_in_origin[2]-path_in_origin[1])*scale_factor) != pix_target:
            #     print(f'{label} is not the correct width, it is {(path_in_origin[2]-path_in_origin[1])*scale_factor} instead of {pix_target}')

            """ this strategy did not work since the svg in the glossary and the svgs in the pages ahve slightly different aspect ratios """
            # x_svg = round(path_in_origin[1]*scale_factor)
            # y_svg = round(path_in_origin[3]*scale_factor)
            # width = round((path_in_origin[2]-path_in_origin[1])*scale_factor)
            # height = round((path_in_origin[4]-path_in_origin[3])*scale_factor)
            # # wrap the svg string in a svg element
            # svg_data = f'<svg xmlns="http://www.w3.org/2000/svg" x="{x_svg}" y="{y_svg}" width="{width}px" height="{height}px" viewBox="{x_svg} {y_svg} {width} {height}"><path d="{svg_string}" fill="none" stroke="#000000" stroke-width="1" /></svg>'

            """ So i will set the canvas to a fixed size of 1000x1000px and center the path in the canvas
                    The width willbe fixed to 500 px and the height will be variable, depending on the aspect ratio of the path """
            width = pix_target * 2
            height = pix_target * 2
            x_svg = pix_target - ((path_in_origin[2]-path_in_origin[1])*scale_factor)/2
            y_svg = pix_target - ((path_in_origin[4]-path_in_origin[3])*scale_factor)/2

            # # used this to figure out the max height of the glossary images
            # pg_height.append(height)

            # print(f'<svg xmlns="http://www.w3.org/2000/svg" x="{min_x}" y="{min_y}" width="{max_x - min_x}px" height="{max_y - min_y}px" viewBox="0 0 0 0"><path d="{svg_string}" fill="#000000" stroke="#000000" stroke-width="0.5" /></svg>')
            # wrap the svg string in a svg element
            svg_string = f'<svg xmlns="http://www.w3.org/2000/svg" width="{width}px" height="{height}px" viewBox="{-x_svg} {-y_svg} {width} {height}"><path d="{svg_path}" fill="#000000" stroke="#000000" stroke-width="1" /></svg>'

            # Convert to raster 
            raster = cairosvg.svg2png(bytestring=svg_string, output_width=width, output_height=height)

            # # save file to disk for visual inspection
            # with open(f"1stSpanish/rasterization/{label}.png","wb") as file:
            #     file.write(raster)
            # with open(f'1stSpanish/rasterization/{label}.svg', 'w') as f:
            #     f.write(svg_string)

            # open in memory file to use for comparison
            raster = Image.open(BytesIO(raster))

            # Convert PIL Images to OpenCV images
            raster= np.array(raster)

            # append the image to the dictionary with the label as key and the nparray as value
            # print('This is the label to append',label)
            gloss_np_dict[label] = raster
    return gloss_np_dict

# # Used this to figure out the max height
# # get max in gloss_height list
# gloss_max_height = max(gloss_height)
# print(gloss_max_height)

# ------------------------------------------------------------------------------------------------------------------------------------------------------------------


# Pages svg
creates a np.array(raster) for the page svg grouped paths, and compares with the gloss np.array(raster) retriving the image label

In [ ]:

def proc_and_compare_pg_svg(path_groups,gloss_np_dict):
    # target size for the images
    pix_target = 500

    ssim_dict = {}
    mse_dict = {}
    pg_height = []
    for page, val in path_groups.items():
        for labeled, groups_dict in val.items():
            for i, path_list in groups_dict.items():

                # flatten the path_elements list
                path_flattened = [val for sublist in path_list for val in sublist]
                # print(flattened)

                """ ---------------------------------------------------------------------- """
                # do not print this section -> svg_string = path.d()
                nullwrite = NullWriter()
                oldstdout = sys.stdout
                sys.stdout = nullwrite # disable output

                # Translate path to origin [0] and get the dim [1:]
                path_in_origin = translate_path_to_origin(path_flattened)
            
                # Scale path
                scale_factor = pix_target/(path_in_origin[2]-path_in_origin[1])
                path_scaled = scale_path(path_in_origin[0], scale_factor, scale_factor)
                sys.stdout = oldstdout # enable output        
                """ ---------------------------------------------------------------------- """

                # Convert list of svgpathtools.path.Line objects to a Path object
                path = Path(*path_scaled)

                # print('get the bbox of the resulting path', get_bbox(path))
                # Convert path to svg string
                svg_path = path.d()


                """ This is the conversion of the page svg string to raster """

                # if (path_in_origin[2]-path_in_origin[1])*scale_factor != 1000:
                #     print(f'{labeled} is not the correct width, it is {(path_in_origin[2]-path_in_origin[1])*scale_factor} instead of 1000')

                """ this strategy did not work since the svg in the glossary and the svgs in the pages ahve slightly different aspect ratios """
                # x_svg = round(path_in_origin[1]*scale_factor)
                # y_svg = round(path_in_origin[3]*scale_factor)
                # width = round((path_in_origin[2]-path_in_origin[1])*scale_factor)
                # height = round((path_in_origin[4]-path_in_origin[3])*scale_factor)
                # # wrap the svg string in a svg element
                # svg_data = f'<svg xmlns="http://www.w3.org/2000/svg" x="{x_svg}" y="{y_svg}" width="{width}px" height="{height}px" viewBox="{x_svg} {y_svg} {width} {height}"><path d="{svg_string}" fill="none" stroke="#000000" stroke-width="1" /></svg>'

                """ So i will set the canvas to a fixed size of 1000x1000px and center the path in the canvas
                        The width will be fixed to 500 px and the height will be variable, depending on the aspect ratio of the path """
                width = pix_target * 2
                height = pix_target * 2
                x_svg = pix_target - ((path_in_origin[2]-path_in_origin[1])*scale_factor)/2
                y_svg = pix_target - ((path_in_origin[4]-path_in_origin[3])*scale_factor)/2

                # # used this to figure out the max height of the glossary images
                # pg_height.append(height)

                # print(f'<svg xmlns="http://www.w3.org/2000/svg" x="{min_x}" y="{min_y}" width="{max_x - min_x}px" height="{max_y - min_y}px" viewBox="0 0 0 0"><path d="{svg_string}" fill="#000000" stroke="#000000" stroke-width="0.5" /></svg>')
                # wrap the svg string in a svg element
                svg_string = f'<svg xmlns="http://www.w3.org/2000/svg" width="{width}px" height="{height}px" viewBox="{-x_svg} {-y_svg} {width} {height}"><path d="{svg_path}" fill="#000000" stroke="#000000" stroke-width="1" /></svg>'

                # Convert to raster with white backgound
                raster = cairosvg.svg2png(bytestring=svg_string, output_width=width, output_height=height)

                # # save file to disk for visual inspection
                # with open(f"1stSpanish/rasterization/{page}_{labeled}_{i}.png","wb") as file:
                #     file.write(raster)
                # with open(f'1stSpanish/rasterization/{page}_{labeled}_{i}.svg', 'w') as f:
                #     f.write(svg_string)

                # open in memory file to use for comparison
                raster = Image.open(BytesIO(raster))

                # Convert PIL Images to OpenCV images
                raster= np.array(raster)

                # # get the dimensions of the raster
                # pg_height, pg_width, _ = raster.shape
                
                # append the image to the dictionary with the label as key and the nparray as value
                # ssim_dict[f'{page}_{labeled}_{i}'] = []
                
                
                for gloss_np_array in gloss_np_dict.items():#gloss_np_array is tupple (label, array)
                    # use open cv to compare the images
                    # reshape = cv2.resize(gloss_np_array[1], (raster.shape[1], raster.shape[0]), interpolation=cv2.INTER_AREA)
                    # dispay = np.hstack((reshape, raster))
                    # cv2.imshow('image', dispay)
                    # cv2.waitKey(0)
                    # cv2.destroyAllWindows()
                    # mean squared error (MSE)
                    mse = np.mean((raster - gloss_np_array[1]) ** 2)

                    # print(f'{page}_{labeled}_{i} vs {gloss_np_array[0]} results', 'mse:', mse)

                    if f'{page}_{labeled}_{i}' not in mse_dict:
                        # print(f'{page}_{labeled} not in mse_dict')
                        mse_dict[f'{page}_{labeled}_{i}'] = [(gloss_np_array[0], mse)]
                    else:
                        # print(f'{page}_{labeled} in mse_dict')
                        mse_dict[f'{page}_{labeled}_{i}'].append((gloss_np_array[0], mse))



                    # # Strutural Similarity Index (SSIM)
                    # # Use a smaller window size
                    # win_size = 3  # or any odd number smaller than your image dimensions
                    # ssim_value = ssim(raster, gloss_np_array[1], multichannel=False, win_size=win_size)

                    # # print(f'{page}_{labeled}_{i} vs {gloss_np_array[0]} results', 'ssim:', ssim_value,"\n")

                    # ssim_dict[f'{page}_{labeled}_{i}'].append((gloss_np_array[0], ssim_value))

                    

                    # # feature similarity index (FSIM) - why is fsim not using gloss_np_array[1] for compare?
                    # fsim = cv2.quality.QualityBRISQUE.compute(raster)
                    # print(f'mse: {mse}, ssim: {ssim}, fsim: {fsim}')
                    # print('-----------------------')
                    # print(f'{page}_{labeled}_{i} vs {key} results', 'mse:', 'mse', 'ssim:', 'ssim', 'fsim:', 'fsim',"\n")

                    """ # feature matching - ORB
                    orb = cv2.ORB_create()
                    kp1, des1 = orb.detectAndCompute(raster,None)
                    kp2, des2 = orb.detectAndCompute(gloss_np_array[1],None)
                    bf = cv2.BFMatcher()
                    matches = bf.knnMatch(des1,des2, k=2)
                    # Apply ratio test
                    good = []
                    for m,n in matches:
                        if m.distance < 0.75*n.distance:
                            good.append([m])
                    print('------ORB results------')
                    print(len(good))
                    print(len(matches))
                    # print(len(good)/len(matches))
                    print('-----------------------') """

                    # # feature matching - SIFT
                    # sift = cv2.SIFT_create()
                    # kp1, des1 = sift.detectAndCompute(raster,None)
                    # kp2, des2 = sift.detectAndCompute(gloss_np_array[1],None)
                    # bf = cv2.BFMatcher()
                    # matches = bf.knnMatch(des1,des2, k=2)
                    # # Apply ratio test
                    # good = []
                    # for m,n in matches:
                    #     if m.distance < 0.75*n.distance:
                    #         good.append([m])
                    # print('------SIFT results------')
                    # print(len(good))
                    # print(len(matches))
                    # print(len(good)/len(matches))
                    # print('-----------------------')

                    # # feature matching - SURF
                    # surf = cv2.SURF_create()
                    # kp1, des1 = surf.detectAndCompute(raster,None)
                    # kp2, des2 = surf.detectAndCompute(gloss_np_array[1],None)
                    # bf = cv2.BFMatcher()
                    # matches = bf.knnMatch(des1,des2, k=2)
                    # # Apply ratio test
                    # good = []
                    # for m,n in matches:
                    #     if m.distance < 0.75*n.distance:
                    #         good.append([m])
                    # print('------SURF results------')
                    # print(len(good))
                    # print(len(matches))
                    # print(len(good)/len(matches))
                    # print('-----------------------')



    # # find the max value for ssim ([1] in each tupple in the list in ssim_dict[f'{page}_{labeled}_{i}']) in each key of ssim_dict and return the label (in [0] in each tupple in the list in ssim_dict[f'{page}_{labeled}_{i}'])
    # for key, value in ssim_dict.items():
    #     if len(value) > 0:
    #         max_value = max(value, key=lambda x: x[1])
    #         print(key, max_value[0], max_value[1])

    """ res_list = []
    for key, value in mse_dict.items():
        if len(value) > 0:
            min_value = min(value, key=lambda x: x[1])
            print(key, min_value[0], min_value[1])
            res_list.append((key, min_value[0], min_value[1])) """

    return mse_dict
# #used this to figure out the max height in pg svgs
# pg_max_height = max(pg_height)
# print(pg_max_height)



# this is the one to run in the sequence
<span>This loops through the pages and gets the text data into the reulsts dict:</span><br>
<span style="color:grey;font-size: 10px">resultDict = {'companies':{'company_name': company_name, 'company_imgs': company_imgs, 'company_description': company_description, 'products':
        {'page': i+1, 'product_name': product_name, 'product_description': product_description, 'scope': [],'destType':[],'solType':[]}}.append({product: [{'page':i+1,'destType':[],'solType':[]}]})</span>
<br>
<span style="color:red"><b>Note: because JSON can't serialize bytes (Object of type bytes is not JSON serializable), the image is saved as string encoded with utf-8. before being used it mus be decoded, the function to encode and decode are as shown:</b>

original_string = 'Hello world'<br>
<b style="color:green">encoded_bytes = original_string.encode('utf-8')<br>
decoded_string = encoded_bytes.decode('utf-8')</b><br>
<br>
print(decoded_string)  # Outputs: Hello world</span>


I am passing not only the bytes, but also the other charateristis in the get_text_dict, so we need to access the 'image' key to decode the image

In [ ]:
import gc
import os
import fitz
from collections import defaultdict
import re

import base64


def join_close_blocks(blocks_in_rect):
    # Initialize the list of new blocks and the previous block
    new_blocks = []
    prev_block = None
    # Iterate over all blocks
    for block in blocks_in_rect:
        # print('prev_block:', prev_block, '\n', 'block:', block, '---------------------------------\n')
        # If there is a previous block and it is close to the current block
        if prev_block is not None and abs(block['bbox'][1] - prev_block['bbox'][3]) < 10:
            # Join the blocks
            prev_block['lines'] += block['lines']
            # Update the bounding box of the previous block
            prev_block['bbox'] = (min(prev_block['bbox'][0], block['bbox'][0]),
                                  min(prev_block['bbox'][1], block['bbox'][1]),
                                  max(prev_block['bbox'][2], block['bbox'][2]),
                                  max(prev_block['bbox'][3], block['bbox'][3]))
        else:
            # Add the previous block to the list of new blocks
            if prev_block is not None:
                new_blocks.append(prev_block)
            # The current block becomes the previous block for the next iteration
            prev_block = block

    # Add the last block to the list of new blocks
    if prev_block is not None:
        new_blocks.append(prev_block)

    return new_blocks



""" In my document unordered lists are identified with '•' (1st level lists), '>'(2nd level lists), or '.' (3rd level lists), at the start of the line in blocks returned by pymupdf Page.get_text('dict')['blocks']
    if span['text'][0] == '•', it sould remain a separate line span
    or
    if span['text'][0] == '>', it sould remain a separate line span. and span['text'][0] = '>' should be replaced with span['text'][0] = '▪'
    if span['text'][:1] == ' .' #and the previous span ends with ':' (previous_span['text'][-1] == ':', i believe),# it sould remain a separate line span, or if span['text'][0] == '.' and previous_span['text'][0] == ('.' or '>' | '▪', or '•'), # it sould remain a separate line span. the span start span['text'][:1] = ' .' in the resulting block should should be replaced with span['text'][0] = '>'.
    the problem with this one is that some text lines are split in two spans, where the first span does not have the period of the sentence, and the second span has the period of the sentence before it at its beggining. can you help me 

    In my document ordered lists are identified span['text'][0].isdigit() and span['text'][1] == '.' at the start of the line
    if span['text'][0] == ' ' and re.match(r'\d+\.', span['text']) or span['text'][0].isdigit() and span['text'][1] == '.', it sould remain a separate line span. """


import re
def create_new_lines(blocks):

    # Iterate over all blocks
    for block in blocks:
        # Iterate over all lines in the block
        new_lines = []
        prev_line = None
        for line in block['lines']:
            # Check if the first span is a list element
            first_span = line['spans'][0]
            is_list_element = (
                first_span['text'][0] == '•' or
                first_span['text'][0] == '>' or 
                (len(first_span['text']) > 1 and first_span['text'][:2] == ' >') or
                first_span['text'][0] == '.' or
                (len(first_span['text']) > 1 and first_span['text'][0] == ' ' and re.match(r'\d+\.', first_span['text'][1:])) or 
                (len(first_span['text']) > 1 and first_span['text'][0].isdigit() and first_span['text'][1] == '.')
            )
            
            """ ---------------------------------------------------------------------- """
            # If the span is a list element, update its text
            if is_list_element:
                if line['spans'][0]['text'][0] == '>':
                    line['spans'][0]['text'] = '▪' + line['spans'][0]['text'][1:]
                elif len(line['spans'][0]['text']) > 1 and line['spans'][0]['text'][:2] == ' >':
                    line['spans'][0]['text'] = '▪' + line['spans'][0]['text'][2:]
                elif line['spans'][0]['text'][0] == '.':
                    line['spans'][0]['text'] = '»' + line['spans'][0]['text'][1:]
            """ ---------------------------------------------------------------------- """

            # I there is a previous line, append the spans to the previous line
            if prev_line is not None:
                prev_line['spans'] += line['spans']
                # Update the 'bbox' of the previous line
                prev_line['bbox'] = (min(prev_line['bbox'][0], line['bbox'][0]), min(prev_line['bbox'][1], line['bbox'][1]), max(prev_line['bbox'][2], line['bbox'][2]), max(prev_line['bbox'][3], line['bbox'][3]))
            else:
                # Otherwise, create a new line and add it to the list of new lines
                new_line = dict(line)
                new_lines.append(new_line)
                prev_line = new_line

        # Replace the old lines with the new lines
        block['lines'] = new_lines

    return blocks


def create_new_spans(blocks):
    # List of sizes to be treated as 10
    sizes_to_be_treated_as_10 = [9.161468505859375, 9.5, 9.746794700622559, 9.899495124816895, 9.949873924255371]

    # font to be treated as Eina01-Light: 'Eina01-Regular'

    # Iterate over all blocks
    for block in blocks:
        # Iterate over all lines in the block
        for line in block['lines']:
            new_spans = []
            prev_span = None

            # Iterate over all spans in the line
            for span in line['spans']:
                # Treat certain sizes as 10
                if span['size'] in sizes_to_be_treated_as_10:
                    span['size'] = 10
                
                if span['font'] == 'Eina01-Regular':
                    # Treat certain fonts as Eina01-Light
                    span['font'] = 'Eina01-Light'

                # Check if the span is a list element
                is_list_element = (
                    span['text'][0] == '•' or
                    span['text'][0] == '▪' or
                    span['text'][0] == '»' or
                    (len(span['text']) > 1 and span['text'][0] == ' ' and re.match(r'\d+\.', span['text'][1:])) or 
                    (len(span['text']) > 1 and span['text'][0].isdigit() and span['text'][1] == '.')

                 
                )

                # If the size, font, and color are the same as the previous span, and the span is not a list element, append the text to the previous span
                if prev_span is not None and span['size'] == prev_span['size'] and span['font'] == prev_span['font'] and span['color'] == prev_span['color'] and not is_list_element:
                    prev_span['text'] += ' ' + span['text']
                    # Update the 'origin' and 'bbox' of the previous span
                    prev_span['origin'] = (min(prev_span['origin'][0], span['origin'][0]), min(prev_span['origin'][1], span['origin'][1]))
                    prev_span['bbox'] = (min(prev_span['bbox'][0], span['bbox'][0]), min(prev_span['bbox'][1], span['bbox'][1]), max(prev_span['bbox'][2], span['bbox'][2]), max(prev_span['bbox'][3], span['bbox'][3]))
                else:
                    # Otherwise, create a new span and add it to the list of new spans
                    new_span = dict(span)
                    new_spans.append(new_span)
                    prev_span = new_span

            # Replace the old spans with the new spans
            line['spans'] = new_spans



            # Update the 'bbox' of the line
            line['bbox'] = (min(span['bbox'][0] for span in line['spans']), min(span['bbox'][1] for span in line['spans']), max(span['bbox'][2] for span in line['spans']), max(span['bbox'][3] for span in line['spans']))

    return blocks

# this returns the html formated text in the rect as a single line of html text, for each block, where the spans are:
# joined into a single line of textx, if they have the same size, font and color,
# encapsulated in <b> tags or if they only differ in the bold font and joined to the text line.
# if the span is a bullet point, it is added to a new line
def get_htmlText_in_rect(page, rect):
    blocks_in_rect = []
    blocks_in_rect_ = []

    # get the blocks in the rect
    blocks_in_rect = get_blocks_in_rect(page, rect)

    # Join close blocks into a single block
    blocks_in_rect0 = join_close_blocks(blocks_in_rect)

    blocks_in_rect1 = create_new_lines(blocks_in_rect0)
                
    # Join all spans with the same size, font, and color into a single span
    blocks_in_rect2 = create_new_spans(blocks_in_rect1)


    # Convert the blocks to text
    blocks_in_rect_text = ''
    for block in blocks_in_rect2:
        # prev_span = None
        # new_line = ''
        for line in block['lines']:
            prev_span = None
            new_line = ''
            for span in line['spans']:
                if len(span['text']) > 1 and (span['text'][0] == '•' or (span['text'][0] == ' ' and re.match(r'\d+\.', span['text']) or span['text'][0].isdigit() and span['text'][1] == '.')):
                    new_line += f'<br>{span["text"]}'
                elif span['text'][0] == '▪': #'>':# this is the sublist bullet point for unordered lists
                    new_line += f'<br>&nbsp;{span["text"]}'
                elif span['text'][0] == '»': #'.':# this is the subsublist bullet point for unordered lists
                    new_line += f'<br>&nbsp;&nbsp;{span["text"]}'
                elif span['font'] == 'Eina01-Bold':
                    new_line += f'<b>{span["text"]}</b>' # handles bold characters
                elif span['font'] == 'Eina01-LightItalic':
                    new_line += f'<i>{span["text"]}</i>' # handles italic characters

                elif span['size'] == 5.829999923706055 and span['origin'][1] > prev_span['origin'][1]:
                    new_line += f'<sub>{span["text"]}</sub>' # handles subscript characters
                elif span['size'] == 5.829999923706055 and span['origin'][1] < prev_span['origin'][1]:
                    new_line += f'<sup>{span["text"]}</sup>' # handles superscript characters
                elif new_line and span['text'] and (new_line[-1].isalnum() or new_line[-1] == '>') and (span['text'][0].isalnum() or span['text'][0] == '>'):
                    new_line += f'<br>{span["text"]}'
                else:
                    new_line += span['text']

                prev_span = span
        blocks_in_rect_text += f'<div>{new_line}</div>'

    return blocks_in_rect_text

def get_blocks_in_rect(page, rect):
    blocks_in_rect = []

    # Iterate over all blocks
    for block in page.get_text('dict')['blocks']:
        # Check if the block is within the rectangle
        if fitz.Rect(block['bbox']).intersects(rect):
            # Copy the block
            block_in_rect = dict(block)
            if 'lines' in block_in_rect:
                # Filter lines within the rectangle
                block_in_rect['lines'] = [line for line in block['lines'] if fitz.Rect(line['bbox']).intersects(rect)]
                # Add the block to the list
                blocks_in_rect.append(block_in_rect)

    return blocks_in_rect

def get_min_values(outer_dict):
    min_values = {}
    for key, value in outer_dict.items():
        min_value = min(value, key=lambda x: x[1])
        min_values[key] = min_value
    return min_values


# Create a dictionary to store the results
resultDict = {'companies': {}}

def process_pdf(file_path, start_page, end_page):
    # Read the PDF file
    doc = fitz.open(file_path)



    counter = 0
    x_max = 0
    not_bbox = []
    product_list = {}
    product_counter = 0

    # ---------------------------------------------------------
    # Get the glossary svg images
    elements_list, gloss_bboxes = process_pdf_gloss_svg(doc, 8)
    # Get the np arrays of the glossary images
    gloss_np_dict = process_glossary_svg(elements_list)
    # ---------------------------------------------------------

    # Iterate through the pages
    for i in range(start_page, end_page):
        page = doc[i]
        company_name = ''
        company_url = ''
        product_name = ''
        company_description = ''
        # company_description_end = 0
        # compDectiption_block = []

        prodDescr_y_min = 0
        com_Descr_y_min = 0
        com_Descr_y_max = 0

        img_list = []

        im_dict = {}
        l=0
        for block in page.get_text('dict')['blocks']:


            # get the images in the page
            # img_list = page.get_images()
            # print('img_list', img_list)
            if block['type'] == 1:
                l += 1

                # get the image (pixmap) from the image_fitz_bboxes
                pix = page.get_pixmap(clip=block['bbox'])
                data = pix.samples
                # create a PIL image from the data
                img = Image.frombytes("RGB", [pix.width, pix.height], data)

                #preview the image
                # img.show()

                # Save the PIL image to a BytesIO stream in PNG format
                img_stream = io.BytesIO()
                img.save(img_stream, format='PNG')

                # Get the image file data as a byte string
                img_data = img_stream.getvalue()

                """i shoul use resnet  to exclude the wrong images"""
                #append the data to the dictionary with the page number as value
                im_dict[img_data] = l

                
                # Append the image data to the list of images
                img_list.append(img_data)
                # print('img_list', img_list)

            if 'lines' in block:
                # print(block['number'], len(block['lines']), block['lines'][0]['spans'][0]['text'])

                for line in block['lines']:

                    for span in line['spans']:

                        if span['size'] > 20 and span['font'] == 'Eina01-Bold' and span['color'] == 14747411:
                            # Check if the last word in product_name ends with a hyphen
                            if product_name and product_name.split()[-1].endswith('-'):
                                # Remove the hyphen and append the next word without a space
                                product_name = product_name.rsplit('-', 1)[0] + span['text']
                            else:
                                product_name += span['text']#' ' + span['text']
                            # get y_mmax of the block where span['size'] > 20 and span['font'] == 'Eina01-Bold' and span['color'] == 14747411
                            prodDescr_y_min = block['bbox'][3]
                        if span['size'] == 10 and span['font'] == 'Eina01-Bold' and span['color'] == 14747411:
                            company_url += span['text']
                            com_Descr_y_max = block['bbox'][1]
                        if span['size'] > 28 and span['font'] == 'Eina01-Bold'and span['color'] == 0:# and len(span['text']) >1:
                            company_name += span['text']
                            com_Descr_y_min = span['bbox'][3]
    
        
        #  Draw a red rectange in each page with x_min = 0, x:max = 375, y_min = (y_min of the block where span['size'] == 10 and span['font'] == 'Eina01-Bold' and span['color'] == 14747411), y_max = 907
        """  This is the company discripton area """
        rect_comp = fitz.Rect(0, com_Descr_y_min, 375, com_Descr_y_max)
        # #  Get all different span sizes in the company description area
        company_description = get_htmlText_in_rect(page, rect_comp)


        #  Draw a blue rectange in each page with x_min = 375, x:max = 1035, y_min = (y_max of the block where span['size'] > 20 and span['font'] == 'Eina01-Bold' and span['color'] == 14747411), y_max = 907
        """  This is the product discripton area """
        rect_prod = fitz.Rect(375, prodDescr_y_min, 1035, 485)
        page.draw_rect(rect_prod, color=(0, 0, 1))       
        product_description = get_htmlText_in_rect(page, rect_prod) 

        # resultDict = {'companies':{'company_name': company_name, 'company_imgs': company_imgs, 'company_description': company_description, 'products':
        # {'page': i+1, 'product_name': product_name, 'product_description': product_description, 'scope': [],'destType':[],'solType':[]}}.append({product: [{'page':i+1,'destType':[],'solType':[]}]})
        # (377, 510, 600, 565)
        rect_scope = fitz.Rect(377, 510, 600, 565)
        scope = get_blocks_in_rect(page, rect_scope)
        scope = [line['spans'][0]['text'] for block in scope for line in block['lines'] for span in line['spans']]
        # print(scope)


        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        # company_imgs = img_list[0]# or company_imgs
        img64_list = []
        for image in im_dict.keys():#img_list:
            # convert the image bytes to base64
            image_base64 = base64.b64encode(image).decode('utf-8')
            img64_list.append(image_base64)
        
        # update the image in the dictionary with the base64 string
        company_imgs = img64_list

        # # convert the image bytes to base64
        # company_imgs_base64 = base64.b64encode(company_imgs).decode('utf-8')#['image']).decode('utf-8')

        # update the image in the dictionary with the base64 string
        # company_imgs = company_imgs_base64# company_imgs['image'] = company_imgs_base64

        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~




        # ------------------------------------------------------------------------------------------------------------------------------------------------------------------
        # Get all SVG paths within the page area for 'dest_type' and 'sol_type' identification
        # --------------
        results_elements = {}
        results_containers = {}
        # do not print this section -> svg_string = path.d()
        nullwrite = NullWriter()
        oldstdout = sys.stdout
        sys.stdout = nullwrite # disable output        
        # Scale path
        result = process_pdf_page_svg(page, i)

        results_elements[i] = result[0][i]
        results_containers[i] = result[1][i]

        # Group the paths in the page within the container rectangles
        path_groups, _ = goup_pg_svgs(results_elements,results_containers)

        sys.stdout = oldstdout # enable output        
        # --------------

        # tranform svg to raster, and to np array, and compare with glossary images
        """ mse_dict[f'{page}_{labeled}_{i}'] """
        comp_vision_res = proc_and_compare_pg_svg(path_groups,gloss_np_dict)

        destType_list = []
        solType_list = []
        min_values = get_min_values(comp_vision_res)
        # print(min_values)

        for key, value in min_values.items():
            if f'{i}_destType' in key:
                destType_list.append(value[0])
            elif f'{i}_solType' in key:
                solType_list.append(value[0])


                    



        
        # ------------------------------------------------------------------------------------------------------------------------------------------------------------------

        # global resultDict
        # Add the company and product to the dictionary
        if company_name in resultDict['companies']:
            # If the dictionary exists, append 'element' to the list
            resultDict['companies'][company_name]['products'].append({
                'page':i+1, 
                'product_name': product_name, 
                'product_description': product_description, 
                'target_audience':'',
                'scope': scope, 
                'destType':destType_list,
                'solType':solType_list})

            resultDict['companies'][company_name]['company_description'] = company_description
            resultDict['companies'][company_name]['company_url'] = company_url
            resultDict['companies'][company_name]['phone'] = ''
            resultDict['companies'][company_name]['email'] = ''
            # resultDict['companies'][company_name]['address'] = ''
            # resultDict['companies'][company_name]['geoRef'] = ''


            resultDict['companies'][company_name]['company_logo'] = ''
            resultDict['companies'][company_name]['company_QR'] = ''

            resultDict['companies'][company_name]['company_imgs'].append(company_imgs)            
        else:
            # If the dictionary doesn't exist, create a new one
            resultDict['companies'][company_name] = {
                'company_description': company_description,
                'company_url': company_url,
                'phone': '',
                'email': '',
                'address':'',
                'geoRef':'',                
                'company_logo': '',
                'company_QR': '',

                'company_imgs': [company_imgs],
                
                'products': [
                    {
                        'page': i+1,
                        'product_name': product_name,
                        'product_description': product_description,
                        'target_audience':'',
                        'scope': scope,
                        'destType': destType_list,
                        'solType': solType_list
                    }
                ]
}

    return resultDict



    # doc.close()
    # del globals()['doc']

result = process_pdf(pdfFile, 11, 263)
print(result)
# Save the results to a JSON file
with open('1stSpanish/result1.json', 'w', encoding='utf-8') as f:
    json.dump(result, f, ensure_ascii=False, indent=4)

# create a graph for the json

In [ ]:
import json
import networkx as nx
import matplotlib.pyplot as plt

# Load your JSON data
with open('1stSpanish/result1.json', encoding='utf-8') as f:
    data = json.load(f)

# Create a directed graph
G = nx.DiGraph()

def add_edges(dat, parent=None):
    if isinstance(dat, dict):
        for k, v in dat.items():
            if parent is not None:
                G.add_edge(parent, k)
            add_edges(v, parent=k)
    elif isinstance(dat, list):
        for i, v in enumerate(dat):
            if parent is not None:
                G.add_edge(parent, f"{parent}_{i}")
            add_edges(v, parent=f"{parent}_{i}")

add_edges(data)

# Draw the graph
nx.draw(G, with_labels=True)
plt.show()

In [ ]:
import fitz
from IPython.display import display
def process_pdf(file_path, start_page, end_page):
    # Read the PDF file
    doc = fitz.open(file_path)

    # Iterate through the pages
    for i in range(start_page, end_page):
        page = doc[i]
        company_name = ''
        company_url = ''
        product_name = ''
        company_description = ''
        # company_description_end = 0
        # compDectiption_block = []

        prodDescr_y_min = 0
        com_Descr_y_min = 0
        com_Descr_y_max = 0

        img_list = []

        im_dict = {}
        l=0
        for block in page.get_text('dict')['blocks']:


            # get the images in the page
            # img_list = page.get_images()
            # print('img_list', img_list)
            if block['type'] == 1:
                # draw a blue rectangle from block['bbox']
                # page.draw_rect(block['bbox'], color=(0, 0, 1))
                pass

            if 'lines' in block:
                # print(block['number'], len(block['lines']), block['lines'][0]['spans'][0]['text'])

                for line in block['lines']:

                    for span in line['spans']:

                        if span['size'] == 10 and span['font'] == 'Eina01-Bold' and span['color'] == 14747411:# --> company_url
                            logo_area_y_min = block['bbox'][3]
                        if span['text'] == 'Collaborating member of the Smart Destinations Network':
                            logo_area_y_max = block['bbox'][1]
                        else:
                            # get the heght of the page
                            pg_height = page.bound().height
                            logo_area_y_max = pg_height

        # get the width of the page
        logo_area_x_max = page.bound().width    
        """  This is the logos area """
        rect_logo = fitz.Rect(0, logo_area_y_min, logo_area_x_max, logo_area_y_max)
        page.draw_rect(rect_logo, color=(0, 1, 0))

        """ drawings = page.get_drawings()


        for path in drawings:
            rect = path['rect']#fitz.Rect(path['rect'])
            page.draw_rect(rect, color=(1, 0, 0), width=2) """


        # if block['type'] == 1 and rect_logo.intersects(block['bbox']):#block['bbox'][3] < logo_area_y_max:#rect_logo.contains(block['bbox']):
        #     page.draw_rect(block['bbox'], color=(1, 0, 0))

        # display the page
        # get the image (pixmap) from the image_fitz_bboxes
        pix = page.get_pixmap()
        data = pix.samples
        # create a PIL image from the data
        img = Image.frombytes("RGB", [pix.width, pix.height], data)
        display(img)






process_pdf(pdfFile


In [ ]:
# Import to Omeka
from OmekaAPI import OmekaAPI
from OmekaAPI import SEGITTUR_CATALOGUE
from OmekaAPI import STT_APPLICATION_TYPE

import io

omeka = OmekaAPI('https://sttobservatory.omeka.net', '2ec26408471c3ea5e76972f76fe14a981214b57d')
# src_pdf = fitz.open("../Catalogues/Catalogue European.pdf")
src_pdf = fitz.open("/Catalogues/Catalogue Spanish.pdf")
# stts_folder = "../EU_2022"
stts_folder = "/Spain"


for company, value in result['companies'].items():
    for product in value['products']:
        initial_pdf_page = product['page']-1
        product_name = '<h1>'+ product['product_name'] + '</h1>'
        product_description = product['product_description']

        # compurl = 'https://'+ value['company_url']

        company_url = [value['company_url']]#f'<a href="{compurl}">{compurl.split("/")[-1]}</a></li>'

        company_imgs_base64 = value['company_imgs']#[0]['image']

        # flatten the list of lists and keep ony unique values
        company_imgs_base64 = [item for sublist in company_imgs_base64 for item in sublist]
        company_imgs_base64 = list(set(company_imgs_base64))

        company_name = company

        company_description = value['company_description']

        img_tag = ''
        for image in company_imgs_base64:
            # print(len(company_imgs_base64),len(image))
            company_img_base64 = image
            #==========================================================
            # Create a data URL for the imge
            data_url = f"data:image/png;base64,{company_img_base64}"

            # Now we can use `data_url` in an img tag
            img_tag = f'<br><img src="{data_url}" alt="Logo">'


        
        creator_description = '<h2>'+ company_name +'</h2>' + company_description + img_tag
            #==========================================================

        print(creator_description)

        page = src_pdf[initial_pdf_page]
        # print(page.get_text('dict')['blocks'][0]['lines'][0]['spans'][0]['text'])




        """ # # Choose between EU_2022 folder or EU_2023 folder
        # for stt_index, stt in enumerate(stts_founded):# aqui eu tenho que alterar para correr dentro do meu dicionário
        #     if stt.city != "": # --->> Aqui também é da estrutura do Diogo

        # stt_pdf = fitz.open()
        #  esta linha está a selecionar as páginas que queemos do pdf, portanto o sttt folder está para os pdf partidos por pág. tenho que ir buscar o pdf ineiro (pág)
        # stt_pdf.insert_pdf(src_pdf, from_page=stt.initial_pdf_page, to_page=stt.final_pdf_page)# nr da pág
        
        stt_name_without_special_chars = product_name.replace("/","").replace("\"","") #stt.name.replace("/","").replace("\"","")
        
        stt_pdf_name = f"{stts_folder}/{initial_pdf_page}_{stt_name_without_special_chars}.pdf".replace(":","")#f"{stts_folder}/{stt.initial_pdf_page}_{stt_name_without_special_chars}.pdf".replace(":","") # <-------------- é aqui!!!; aqui basta só o nr da pág, antes do '_'
                
        # stt_pdf.save(stt_pdf_name) """


        # Create a new PDF in memory
        new_pdf = fitz.open()
        new_pdf.insert_pdf(src_pdf, from_page=initial_pdf_page, to_page=initial_pdf_page)

        # Write the new PDF to a BytesIO stream
        pdf_stream = io.BytesIO()
        new_pdf.save(pdf_stream)

        # Get the PDF data as a byte string
        pdf_data = pdf_stream.getvalue()

    
        item_id = omeka.post_item(
            title=product_name+'_v1',#results['companies'][keys]['products']['product_name']
            description=product_description,#stt.description,
            creator=creator_description,
            item_type=STT_APPLICATION_TYPE,
            urls=company_url,#stt.urls,# actualizar o dicionário
            collection=omeka.get_collection_id_by_name(SEGITTUR_CATALOGUE)#,
            # address=stt.city
        )
    
        """ for index, image in enumerate(stt.images):
            omeka.post_file_for_item(item_id, f"{stt.name}_{index}.jpg", image)# posso usar isto results[''companies'][keys]['products']['product_name'] como nome da imagem
         """
        """ with open(stt_pdf_name, 'rb') as f:
            omeka.post_file_for_item(item_id, stt_pdf_name, f.read())# dá o pst ao pdf """
        # Send the PDF data to Omeka
        omeka.post_file_for_item(item_id, f"{initial_pdf_page+1}_{product_name}.pdf", pdf_data)
    
        # try:
        #     omeka.post_geolocation_for_item(stt.city,item_id)# no escoces tenho que tratar disso
        # except AttributeError:
        #     continue

In [ ]:
print('done')